Model Generation


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Step 1: Load the dataset
file_path = './Badminton Dataset for Win Predictor - Sheet1.csv'  # Replace with the path to your dataset
dataset = pd.read_csv(file_path)

# Step 2: Data Cleaning
# Drop irrelevant columns
dataset_cleaned = dataset.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'])

# Drop rows with missing critical values
dataset_cleaned = dataset_cleaned.dropna(subset=['SERVER', 'RECIEVER', 'SCOREDBY', 'SCORE', 'OPPONENT SCORE'])

# Convert scores to integers
dataset_cleaned['SCORE'] = dataset_cleaned['SCORE'].astype(int)
dataset_cleaned['OPPONENT SCORE'] = dataset_cleaned['OPPONENT SCORE'].astype(int)

# Standardize scores from a consistent perspective based on the point win logic
def standardize_scores(row):
    if row['SERVER'] == row['SCOREDBY']:
        # If the server wins, no score swap
        return row['SCORE'], row['OPPONENT SCORE']
    else:
        # If the receiver wins, swap the scores
        return row['OPPONENT SCORE'], row['SCORE']

# Apply the standardization function to create new columns
dataset_cleaned[['STANDARDIZED_SCORE', 'STANDARDIZED_OPPONENT_SCORE']] = dataset_cleaned.apply(
    lambda row: pd.Series(standardize_scores(row)), axis=1
)

# Create additional features
dataset_cleaned['POINT_DIFFERENCE'] = dataset_cleaned['STANDARDIZED_SCORE'] - dataset_cleaned['STANDARDIZED_OPPONENT_SCORE']
dataset_cleaned['CUMULATIVE_SCORE'] = dataset_cleaned.groupby(['SCORE_SHEET_ID', 'SCOREDBY']).cumcount() + 1

# Step 3: Define the target variable (WIN) from final records of each set
final_records = dataset_cleaned.groupby(['SCORE_SHEET_ID', 'SET_NO']).tail(1).copy()  # Explicitly create a copy
final_records['WIN'] = (final_records['STANDARDIZED_SCORE'] > final_records['STANDARDIZED_OPPONENT_SCORE']).astype(int)

# Step 4: Prepare features and target for modeling
features = [
    'STANDARDIZED_SCORE',
    'STANDARDIZED_OPPONENT_SCORE',
    'POINT_DIFFERENCE',
    'CUMULATIVE_SCORE'
]
X = final_records[features]
y = final_records['WIN']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train a Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Step 6: Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Display evaluation results
print(f"Model Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

# Step 7: Save the model
model_path = '/content/badminton_win_predictor.pkl'  # Specify the file path
joblib.dump(rf_model, model_path)
print(f"Model saved to: {model_path}")


<ipython-input-14-29175f785df2>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cleaned['SCORE'] = dataset_cleaned['SCORE'].astype(int)
<ipython-input-14-29175f785df2>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cleaned['OPPONENT SCORE'] = dataset_cleaned['OPPONENT SCORE'].astype(int)


Model Accuracy: 1.00
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        72
           1       1.00      1.00      1.00        51

    accuracy                           1.00       123
   macro avg       1.00      1.00      1.00       123
weighted avg       1.00      1.00      1.00       123

Model saved to: /content/badminton_win_predictor.pkl



Proper Calculation Of  Score







In [ ]:

import pandas as pd
import joblib

# Step 1: Load the pre-trained model
model_path = '/content/badminton_win_predictor.pkl'  # Specify the correct path
rf_model = joblib.load(model_path)

# Step 2: Define a function for real-time prediction based on a CSV input file
def standardize_scores(row):
    """
    Standardizes scores such that Player A (fixed perspective) is the scorer.
    If SERVER aligns with SCOREDBY, keep scores as-is.
    Otherwise, swap SCORE and OPPONENT SCORE.
    """
    if row['SERVER'] == row['SCOREDBY']:
        return row['SCORE'], row['OPPONENT SCORE']
    else:
        return row['OPPONENT SCORE'], row['SCORE']

def preprocess_and_predict(csv_path):
    """
    Preprocesses the input data from the CSV file and predicts the match outcomes.
    Args:
    - csv_path: path to the CSV file containing match data

    Returns:
    - DataFrame with predictions and probability of winning for both players
    """
    # Load the dataset
    input_data = pd.read_csv(csv_path)

    # Clean and preprocess the dataset
    input_data[['STANDARDIZED_SCORE', 'STANDARDIZED_OPPONENT_SCORE']] = input_data.apply(
        lambda row: pd.Series(standardize_scores(row)), axis=1
    )

    # Calculate point difference and cumulative score
    input_data['POINT_DIFFERENCE'] = input_data['STANDARDIZED_SCORE'] - input_data['STANDARDIZED_OPPONENT_SCORE']
    input_data['CUMULATIVE_SCORE'] = input_data.groupby(['SCORE_SHEET_ID', 'SCOREDBY']).cumcount() + 1

    # Prepare the features
    features = ['STANDARDIZED_SCORE', 'STANDARDIZED_OPPONENT_SCORE', 'POINT_DIFFERENCE', 'CUMULATIVE_SCORE']
    prediction_data = input_data[features]

    # Get probabilities for each match (class 0 and class 1)
    probabilities = rf_model.predict_proba(prediction_data)

    # Add the predictions and the probability of winning for both players
    input_data['PREDICTION'] = rf_model.predict(prediction_data)
    input_data['RESULT'] = input_data['PREDICTION'].apply(lambda x: "Win" if x == 1 else "Loss")

    # Probability columns: Probabilities of player 1 (Win) and player 0 (Loss)
    input_data['PROB_PLAYER_1'] = probabilities[:, 1]  # Probability for Player 1 (win)
    input_data['PROB_PLAYER_0'] = probabilities[:, 0]  # Probability for Player 0 (loss)

    # Live score columns
    input_data['LIVE_SCORE'] = input_data['STANDARDIZED_SCORE'].astype(str) + " - " + input_data['STANDARDIZED_OPPONENT_SCORE'].astype(str)

    # Server information: Who is serving at the moment
    input_data['SERVER'] = input_data['SERVER']  # This is the current server at that point in the match

    # Step 3: Display predictions with the server and live score
    result_df = input_data[['SCORE_SHEET_ID', 'SET_NO', 'SERVER', 'RECIEVER', 'LIVE_SCORE', 'PROB_PLAYER_1', 'PROB_PLAYER_0', 'PREDICTION', 'RESULT']]

    return result_df

# Example: Predict the outcomes of the matches in a CSV file
csv_path = '/content/game 1.csv'  # Replace with your actual file path

# Call the function and display the results
predicted_results = preprocess_and_predict(csv_path)
print(predicted_results)


    SCORE_SHEET_ID  SET_NO                 SERVER               RECIEVER  \
0                1       1  K.Siddharthan (COSMO)      Aadhavan (SANDYS)   
1                1       1  K.Siddharthan (COSMO)      Aadhavan (SANDYS)   
2                1       1      Aadhavan (SANDYS)  K.Siddharthan (COSMO)   
3                1       1  K.Siddharthan (COSMO)      Aadhavan (SANDYS)   
4                1       1      Aadhavan (SANDYS)  K.Siddharthan (COSMO)   
5                1       1      Aadhavan (SANDYS)  K.Siddharthan (COSMO)   
6                1       1      Aadhavan (SANDYS)  K.Siddharthan (COSMO)   
7                1       1      Aadhavan (SANDYS)  K.Siddharthan (COSMO)   
8                1       1  K.Siddharthan (COSMO)      Aadhavan (SANDYS)   
9                1       1      Aadhavan (SANDYS)  K.Siddharthan (COSMO)   
10               1       1      Aadhavan (SANDYS)  K.Siddharthan (COSMO)   
11               1       1  K.Siddharthan (COSMO)      Aadhavan (SANDYS)   
12          

Simulation and Calculation of Prop.


In [ ]:
import pandas as pd
import time

# Function to standardize the scores (swap if receiver wins)
def standardize_scores(row):
    if row['SERVER'] == row['SCOREDBY']:
        # If the server wins, no score swap
        return row['SCORE'], row['OPPONENT SCORE']
    else:
        # If the receiver wins, swap the scores
        return row['OPPONENT SCORE'], row['SCORE']

# Function to predict winner probabilities based on points won
def predict_winner_probability(player1_score, player2_score):
    # Calculate the total points played so far
    total_points = player1_score + player2_score
    if total_points == 0:
        return 50, 50  # If no points played, 50% chance each

    # Basic heuristic: probability based on points won
    p1_win_prob = (player1_score / total_points) * 100
    p2_win_prob = 100 - p1_win_prob

    return p1_win_prob, p2_win_prob

# Function to simulate the game live with predictions
def simulate_game_with_predictions(csv_file):
    # Read the CSV data
    df = pd.read_csv(csv_file)

    # Ensure we have all necessary columns
    required_columns = ['SCORE_SHEET_ID', 'SET_NO', 'SERVER', 'RECIEVER', 'SCOREDBY', 'SCORE', 'TIME', 'OPPONENT SCORE']
    for col in required_columns:
        if col not in df.columns:
            print(f"Missing column: {col}")
            return

    # Initialize scores for both players
    player1_score = 0
    player2_score = 0
    server = None
    receiver = None

    # Loop through each row and simulate the game live
    for index, row in df.iterrows():
        # Extract relevant data from the row
        score = row['SCORE']
        opponent_score = row['OPPONENT SCORE']
        server = row['SERVER']
        receiver = row['RECIEVER']
        score_by = row['SCOREDBY']
        time_stamp = row['TIME']

        # Convert the time format to match "00:02:00"
        time_parts = time_stamp.split(':')
        formatted_time = f"00:{time_parts[0]}:{time_parts[1]}" if len(time_parts) == 2 else f"00:00:{time_parts[0]}"

        # Standardize the scores using the helper function
        player1_score, player2_score = standardize_scores(row)

        # Predict winner probabilities
        player1_prob, player2_prob = predict_winner_probability(player1_score, player2_score)

        # Print the current score, who scored, and probabilities in the desired format
        print(f"Time: {formatted_time} | Server: {server} | Receiver: {receiver}")
        print(f"Score: {server} {player1_score} - {receiver} {player2_score}")
        print(f"{score_by} scored the point")
        print(f"{server} win probability: {player1_prob:.2f}% | {receiver} win probability: {player2_prob:.2f}%\n")

        # Sleep for a short time to simulate real-time progress
        time.sleep(1)

# Example usage
simulate_game_with_predictions('game 1.csv')  # Replace with your CSV file path


Time: 00:08:32 | Server: K.Siddharthan (COSMO) | Receiver: Aadhavan (SANDYS)
Score: K.Siddharthan (COSMO) 0 - Aadhavan (SANDYS) 0
K.Siddharthan (COSMO) scored the point
K.Siddharthan (COSMO) win probability: 50.00% | Aadhavan (SANDYS) win probability: 50.00%

Time: 00:08:33 | Server: K.Siddharthan (COSMO) | Receiver: Aadhavan (SANDYS)
Score: K.Siddharthan (COSMO) 0 - Aadhavan (SANDYS) 1
Aadhavan (SANDYS) scored the point
K.Siddharthan (COSMO) win probability: 0.00% | Aadhavan (SANDYS) win probability: 100.00%

Time: 00:08:33 | Server: Aadhavan (SANDYS) | Receiver: K.Siddharthan (COSMO)
Score: Aadhavan (SANDYS) 1 - K.Siddharthan (COSMO) 1
K.Siddharthan (COSMO) scored the point
Aadhavan (SANDYS) win probability: 50.00% | K.Siddharthan (COSMO) win probability: 50.00%

Time: 00:08:33 | Server: K.Siddharthan (COSMO) | Receiver: Aadhavan (SANDYS)
Score: K.Siddharthan (COSMO) 1 - Aadhavan (SANDYS) 2
Aadhavan (SANDYS) scored the point
K.Siddharthan (COSMO) win probability: 33.33% | Aadhavan (S

Test On New Dataset

In [ ]:
import pandas as pd
import time

# Function to standardize the scores (swap if receiver wins)
def standardize_scores(row):
    if row['SERVER'] == row['SCOREDBY']:
        # If the server wins, no score swap
        return row['SCORE'], row['OPPONENT SCORE']
    else:
        # If the receiver wins, swap the scores
        return row['OPPONENT SCORE'], row['SCORE']

# Function to predict winner probabilities based on points won, closeness to 30, and momentum
def predict_winner_probability(player1_score, player2_score, momentum_factor):
    # Calculate the total points played so far
    total_points = player1_score + player2_score
    if total_points == 0:
        return 50, 50  # If no points played, 50% chance each

    # Probability based on score closeness to 30
    p1_closeness = player1_score / 30
    p2_closeness = player2_score / 30

    # Adjust probabilities based on momentum factor
    p1_win_prob = p1_closeness * 100 + momentum_factor
    p2_win_prob = p2_closeness * 100 - momentum_factor

    # Limit probabilities to 0-100 range
    p1_win_prob = max(0, min(100, p1_win_prob))
    p2_win_prob = max(0, min(100, p2_win_prob))

    return p1_win_prob, p2_win_prob

# Function to simulate the game live with predictions
def simulate_game_with_predictions(csv_file):
    # Read the CSV data
    df = pd.read_csv(csv_file)

    # Ensure we have all necessary columns
    required_columns = ['SCORE_SHEET_ID', 'SET_NO', 'SERVER', 'RECIEVER', 'SCOREDBY', 'SCORE', 'TIME', 'OPPONENT SCORE']
    for col in required_columns:
        if col not in df.columns:
            print(f"Missing column: {col}")
            return

    # Initialize scores for both players
    player1_score = 0
    player2_score = 0
    server = None
    receiver = None
    momentum_player1 = 0
    momentum_player2 = 0

    # Loop through each row and simulate the game live
    for index, row in df.iterrows():
        # Extract relevant data from the row
        score = row['SCORE']
        opponent_score = row['OPPONENT SCORE']
        server = row['SERVER']
        receiver = row['RECIEVER']
        score_by = row['SCOREDBY']
        time_stamp = row['TIME']

        # Convert the time format to match "00:02:00"
        time_parts = time_stamp.split(':')
        formatted_time = f"00:{time_parts[0]}:{time_parts[1]}" if len(time_parts) == 2 else f"00:00:{time_parts[0]}"

        # Standardize the scores using the helper function
        player1_score, player2_score = standardize_scores(row)

        # Update momentum based on who scored the point
        if score_by == server:
            # Player 1 (server) wins a point
            momentum_player1 += 1
            momentum_player2 = 0  # Reset opponent's momentum
        else:
            # Player 2 (receiver) wins a point
            momentum_player2 += 1
            momentum_player1 = 0  # Reset opponent's momentum

        # Predict winner probabilities with momentum and score closeness
        player1_prob, player2_prob = predict_winner_probability(player1_score, player2_score, momentum_player1 - momentum_player2)

        # Check if a player has won the game (30 points and at least 2 points lead)
        if player1_score >= 30 and player1_score - player2_score >= 2:
            print(f"{server} wins with score: {server} {player1_score} - {receiver} {player2_score}")
            break
        elif player2_score >= 30 and player2_score - player1_score >= 2:
            print(f"{receiver} wins with score: {server} {player1_score} - {receiver} {player2_score}")
            break

        # Print the current score, who scored, and probabilities in the desired format
        print(f"Time: {formatted_time} | Server: {server} | Receiver: {receiver}")
        print(f"Score: {server} {player1_score} - {receiver} {player2_score}")
        print(f"{score_by} scored the point")
        print(f"{server} win probability: {player1_prob:.2f}% | {receiver} win probability: {player2_prob:.2f}%\n")

        # Sleep for a short time to simulate real-time progress
        time.sleep(1)

# Example usage
simulate_game_with_predictions('game 2.csv')  # Replace with your CSV file path


Time: 00:08:47 | Server: Maruthu (V SPORTS) | Receiver: Varshith (COSMO)
Score: Maruthu (V SPORTS) 0 - Varshith (COSMO) 0
Maruthu (V SPORTS) scored the point
Maruthu (V SPORTS) win probability: 50.00% | Varshith (COSMO) win probability: 50.00%

Time: 00:08:48 | Server: Maruthu (V SPORTS) | Receiver: Varshith (COSMO)
Score: Maruthu (V SPORTS) 0 - Varshith (COSMO) 1
Varshith (COSMO) scored the point
Maruthu (V SPORTS) win probability: 0.00% | Varshith (COSMO) win probability: 4.33%

Time: 00:08:48 | Server: Varshith (COSMO) | Receiver: Maruthu (V SPORTS)
Score: Varshith (COSMO) 2 - Maruthu (V SPORTS) 0
Varshith (COSMO) scored the point
Varshith (COSMO) win probability: 7.67% | Maruthu (V SPORTS) win probability: 0.00%

Time: 00:08:48 | Server: Varshith (COSMO) | Receiver: Maruthu (V SPORTS)
Score: Varshith (COSMO) 2 - Maruthu (V SPORTS) 1
Maruthu (V SPORTS) scored the point
Varshith (COSMO) win probability: 5.67% | Maruthu (V SPORTS) win probability: 4.33%

Time: 00:08:49 | Server: Marut

In [ ]:
import pandas as pd
import time

# Function to standardize the scores (swap if receiver wins)
def standardize_scores(row):
    if row['SERVER'] == row['SCOREDBY']:
        # If the server wins, no score swap
        return row['SCORE'], row['OPPONENT SCORE']
    else:
        # If the receiver wins, swap the scores
        return row['OPPONENT SCORE'], row['SCORE']

# Function to predict winner probabilities based on points won, closeness to 30, and momentum
def predict_winner_probability(player1_score, player2_score, momentum_factor):
    # Calculate the total points played so far
    total_points = player1_score + player2_score
    if total_points == 0:
        return 50, 50  # If no points played, 50% chance each

    # Probability based on score closeness to 30
    p1_closeness = player1_score / 30
    p2_closeness = player2_score / 30

    # Adjust probabilities based on momentum factor
    p1_win_prob = p1_closeness * 100 + momentum_factor
    p2_win_prob = p2_closeness * 100 - momentum_factor

    # Limit probabilities to 0-100 range
    p1_win_prob = max(0, min(100, p1_win_prob))
    p2_win_prob = max(0, min(100, p2_win_prob))

    # Adjust so that the sum of probabilities equals 100
    total_prob = p1_win_prob + p2_win_prob
    p1_win_prob = (p1_win_prob / total_prob) * 100
    p2_win_prob = 100 - p1_win_prob  # Ensure total equals 100%

    return p1_win_prob, p2_win_prob

# Function to simulate the game live with predictions
def simulate_game_with_predictions(csv_file):
    # Read the CSV data
    df = pd.read_csv(csv_file)

    # Ensure we have all necessary columns
    required_columns = ['SCORE_SHEET_ID', 'SET_NO', 'SERVER', 'RECIEVER', 'SCOREDBY', 'SCORE', 'TIME', 'OPPONENT SCORE']
    for col in required_columns:
        if col not in df.columns:
            print(f"Missing column: {col}")
            return

    # Initialize scores for both players
    player1_score = 0
    player2_score = 0
    server = None
    receiver = None
    momentum_player1 = 0
    momentum_player2 = 0

    # Loop through each row and simulate the game live
    for index, row in df.iterrows():
        # Extract relevant data from the row
        score = row['SCORE']
        opponent_score = row['OPPONENT SCORE']
        server = row['SERVER']
        receiver = row['RECIEVER']
        score_by = row['SCOREDBY']
        time_stamp = row['TIME']

        # Convert the time format to match "00:02:00"
        time_parts = time_stamp.split(':')
        formatted_time = f"00:{time_parts[0]}:{time_parts[1]}" if len(time_parts) == 2 else f"00:00:{time_parts[0]}"

        # Standardize the scores using the helper function
        player1_score, player2_score = standardize_scores(row)

        # Update momentum based on who scored the point
        if score_by == server:
            # Player 1 (server) wins a point
            momentum_player1 += 1
            momentum_player2 = 0  # Reset opponent's momentum
        else:
            # Player 2 (receiver) wins a point
            momentum_player2 += 1
            momentum_player1 = 0  # Reset opponent's momentum

        # Predict winner probabilities with momentum and score closeness
        player1_prob, player2_prob = predict_winner_probability(player1_score, player2_score, momentum_player1 - momentum_player2)

        # Check if a player has won the game (30 points and at least 2 points lead)
        if player1_score >= 30 and player1_score - player2_score >= 2:
            print(f"{server} wins with score: {server} {player1_score} - {receiver} {player2_score}")
            break
        elif player2_score >= 30 and player2_score - player1_score >= 2:
            print(f"{receiver} wins with score: {server} {player1_score} - {receiver} {player2_score}")
            break

        # Print the current score, who scored, and probabilities in the desired format
        print(f"Time: {formatted_time} | Server: {server} | Receiver: {receiver}")
        print(f"Score: {server} {player1_score} - {receiver} {player2_score}")
        print(f"{score_by} scored the point")
        print(f"{server} win probability: {player1_prob:.2f}% | {receiver} win probability: {player2_prob:.2f}%")
        print(f"Complementary: {receiver} win probability: {100 - player1_prob:.2f}% | {server} win probability: {100 - player2_prob:.2f}%\n")

        # Sleep for a short time to simulate real-time progress
        time.sleep(1)

# Example usage
simulate_game_with_predictions('game 2.csv')  # Replace with your CSV file path


Time: 00:08:47 | Server: Maruthu (V SPORTS) | Receiver: Varshith (COSMO)
Score: Maruthu (V SPORTS) 0 - Varshith (COSMO) 0
Maruthu (V SPORTS) scored the point
Maruthu (V SPORTS) win probability: 50.00% | Varshith (COSMO) win probability: 50.00%
Complementary: Varshith (COSMO) win probability: 50.00% | Maruthu (V SPORTS) win probability: 50.00%

Time: 00:08:48 | Server: Maruthu (V SPORTS) | Receiver: Varshith (COSMO)
Score: Maruthu (V SPORTS) 0 - Varshith (COSMO) 1
Varshith (COSMO) scored the point
Maruthu (V SPORTS) win probability: 0.00% | Varshith (COSMO) win probability: 100.00%
Complementary: Varshith (COSMO) win probability: 100.00% | Maruthu (V SPORTS) win probability: 0.00%

Time: 00:08:48 | Server: Varshith (COSMO) | Receiver: Maruthu (V SPORTS)
Score: Varshith (COSMO) 2 - Maruthu (V SPORTS) 0
Varshith (COSMO) scored the point
Varshith (COSMO) win probability: 100.00% | Maruthu (V SPORTS) win probability: 0.00%
Complementary: Maruthu (V SPORTS) win probability: 0.00% | Varshith 

New formula alpha * score_diff + beta * momentum_factor

In [ ]:
import pandas as pd
import time
import numpy as np
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from collections import deque

# Function to standardize the scores (swap if receiver wins)
def standardize_scores(row):
    if row['SERVER'] == row['SCOREDBY']:
        return row['SCORE'], row['OPPONENT SCORE']
    else:
        return row['OPPONENT SCORE'], row['SCORE']

# Improved function to predict winner probabilities
def predict_winner_probability_improved(player1_score, player2_score, momentum_factor, alpha=0.05, beta=0.02):
    if player1_score == 0 and player2_score == 0:
        return 50.0, 50.0  # Equal probabilities
    score_diff = player1_score - player2_score
    exponent = alpha * score_diff + beta * momentum_factor
    p1_win_prob = 1 / (1 + np.exp(-exponent))  # Logistic function
    p1_win_prob *= 100  # Convert to percentage
    p2_win_prob = 100 - p1_win_prob
    return p1_win_prob, p2_win_prob

# Class for handling file updates
class CSVUpdateHandler(FileSystemEventHandler):
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self.last_row_count = 0
        self.player1_score = 0
        self.player2_score = 0
        self.recent_points = deque(maxlen=5)  # Track last 5 points for momentum

    def process_new_data(self, new_data):
        for _, row in new_data.iterrows():
            server = row['SERVER']
            receiver = row['RECIEVER']
            score_by = row['SCOREDBY']
            time_stamp = row['TIME']

            # Standardize scores
            self.player1_score, self.player2_score = standardize_scores(row)

            # Update recent points (1 for server, -1 for receiver)
            if score_by == server:
                self.recent_points.append(1)  # Server scored
            else:
                self.recent_points.append(-1)  # Receiver scored

            # Compute momentum as the sum of recent points
            momentum = sum(self.recent_points)

            # Predict winner probabilities
            player1_prob, player2_prob = predict_winner_probability_improved(
                self.player1_score, self.player2_score, momentum
            )

            # Print the current score, predictions, and momentum
            print(f"Time: {time_stamp} | Server: {server} | Receiver: {receiver}")
            print(f"Score: {server} {self.player1_score} - {receiver} {self.player2_score}")
            print(f"{score_by} scored the point")
            print(f"Momentum: {momentum} (last {len(self.recent_points)} points)")
            print(f"{server} win probability: {player1_prob:.2f}% | {receiver} win probability: {player2_prob:.2f}%\n")

            # Check if a player wins the game
            if self.player1_score >= 30 and self.player1_score - self.player2_score >= 2:
                print(f"{server} wins with score: {server} {self.player1_score} - {receiver} {self.player2_score}\n")
                break
            elif self.player2_score >= 30 and self.player2_score - self.player1_score >= 2:
                print(f"{receiver} wins with score: {server} {self.player1_score} - {receiver} {self.player2_score}\n")
                break

    def on_modified(self, event):
        if event.src_path.endswith(self.csv_file):
            try:
                # Read the CSV file
                df = pd.read_csv(self.csv_file)

                # Ensure the CSV has the correct structure
                required_columns = ['SCORE_SHEET_ID', 'SET_NO', 'SERVER', 'RECIEVER', 'SCOREDBY', 'SCORE', 'TIME', 'OPPONENT SCORE']
                if not all(col in df.columns for col in required_columns):
                    print("CSV file is missing required columns. Please check the file format.")
                    return

                # Select only new rows
                new_rows = df.iloc[self.last_row_count:]

                if not new_rows.empty:
                    print("\nNew data detected. Processing...\n")
                    self.process_new_data(new_rows)
                    self.last_row_count = len(df)  # Update the row count
            except Exception as e:
                print(f"Error processing file: {e}")

# Function to monitor the CSV file
def monitor_csv_file(csv_file):
    event_handler = CSVUpdateHandler(csv_file)
    observer = Observer()
    observer.schedule(event_handler, path=".", recursive=False)  # Monitor current directory
    observer.start()
    print(f"Monitoring {csv_file} for updates...\n")
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

# Run the script
if __name__ == "__main__":
    csv_file = "game 1.csv"  # Replace with your CSV file name
    monitor_csv_file(csv_file)


Time: 00:08:47 | Server: Maruthu (V SPORTS) | Receiver: Varshith (COSMO)
Score: Maruthu (V SPORTS) 0 - Varshith (COSMO) 0
Maruthu (V SPORTS) scored the point
Maruthu (V SPORTS) win probability: 50.00% | Varshith (COSMO) win probability: 50.00%
Complementary: Varshith (COSMO) win probability: 50.00% | Maruthu (V SPORTS) win probability: 50.00%

Time: 00:08:48 | Server: Maruthu (V SPORTS) | Receiver: Varshith (COSMO)
Score: Maruthu (V SPORTS) 0 - Varshith (COSMO) 1
Varshith (COSMO) scored the point
Maruthu (V SPORTS) win probability: 0.00% | Varshith (COSMO) win probability: 100.00%
Complementary: Varshith (COSMO) win probability: 100.00% | Maruthu (V SPORTS) win probability: 0.00%

Time: 00:08:48 | Server: Varshith (COSMO) | Receiver: Maruthu (V SPORTS)
Score: Varshith (COSMO) 2 - Maruthu (V SPORTS) 0
Varshith (COSMO) scored the point
Varshith (COSMO) win probability: 100.00% | Maruthu (V SPORTS) win probability: 0.00%
Complementary: Maruthu (V SPORTS) win probability: 0.00% | Varshith 

Finetuning of alpha and beta

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# Function to standardize the scores (swap if receiver wins)
def standardize_scores(row):
    try:
        # Attempt to convert scores to integers
        player1_score = int(row['SCORE'])
        player2_score = int(row['OPPONENT SCORE'])
    except ValueError:
        # If conversion fails (e.g., the value is '13:32'), print and skip this row
        print(f"Skipping invalid data: {row['SCORE']} and {row['OPPONENT SCORE']}")
        return None, None  # Return None if invalid data

    if row['SERVER'] == row['SCOREDBY']:
        return player1_score, player2_score
    else:
        return player2_score, player1_score

# Function to clean non-numeric data in SCORE and OPPONENT SCORE columns
def clean_data(df):
    # Convert SCORE and OPPONENT SCORE to numeric, forcing errors to NaN
    df['SCORE'] = pd.to_numeric(df['SCORE'], errors='coerce')
    df['OPPONENT SCORE'] = pd.to_numeric(df['OPPONENT SCORE'], errors='coerce')

    # Drop rows where either score is NaN (invalid data)
    df.dropna(subset=['SCORE', 'OPPONENT SCORE'], inplace=True)
    return df

# Improved function to predict winner probabilities
def predict_winner_probability_improved(player1_score, player2_score, momentum_factor, alpha=0.1, beta=0.05):
    score_diff = player1_score - player2_score
    exponent = alpha * score_diff + beta * momentum_factor
    p1_win_prob = 1 / (1 + np.exp(-exponent))  # Logistic function
    p1_win_prob *= 100  # Convert to percentage
    p2_win_prob = 100 - p1_win_prob
    return p1_win_prob, p2_win_prob

# Function to fine-tune alpha and beta using grid search
def fine_tune_alpha_beta(df):
    # Define the grid of possible alpha and beta values
    alpha_values = [0.05, 0.1, 0.15]
    beta_values = [0.02, 0.05, 0.08]

    best_alpha = None
    best_beta = None
    best_mse = float('inf')  # Start with a large error value

    # Loop over all combinations of alpha and beta
    for alpha in alpha_values:
        for beta in beta_values:
            predicted_probs = []
            actual_results = []

            # Loop over the rows in the dataset
            for _, row in df.iterrows():
                player1_score, player2_score = standardize_scores(row)

                # Skip rows with invalid scores
                if player1_score is None or player2_score is None:
                    continue

                # Assume momentum_factor is calculated here (e.g., from previous wins)
                momentum_factor = 0  # Placeholder for actual momentum calculation

                # Get predicted probabilities
                p1_win_prob, p2_win_prob = predict_winner_probability_improved(
                    player1_score, player2_score, momentum_factor, alpha, beta
                )

                # Actual result (1 if player 1 wins, 0 if player 2 wins)
                actual_result = 1 if row['SCOREDBY'] == row['SERVER'] else 0
                predicted_results = 1 if p1_win_prob > p2_win_prob else 0

                predicted_probs.append(predicted_results)
                actual_results.append(actual_result)

            # Calculate Mean Squared Error (MSE)
            mse = mean_squared_error(actual_results, predicted_probs)

            # Update best alpha and beta if the current MSE is better
            if mse < best_mse:
                best_mse = mse
                best_alpha = alpha
                best_beta = beta

    print(f"Best Alpha: {best_alpha}, Best Beta: {best_beta} with MSE: {best_mse}")
    return best_alpha, best_beta

# Example to load your CSV and clean the data
df = pd.read_csv("Badminton Dataset for Win Predictor - Sheet1.csv")  # Replace with your actual CSV file

# Clean the data by converting non-numeric values
df = clean_data(df)

# Now, the dataframe should contain only valid numeric values for scores
print(df.head())  # Check if the data is clean

# Call the fine-tuning function
best_alpha, best_beta = fine_tune_alpha_beta(df)


   SCORE_SHEET_ID  SET_NO                 SERVER               RECIEVER  \
0               1       1  K.Siddharthan (COSMO)      Aadhavan (SANDYS)   
1               1       1  K.Siddharthan (COSMO)      Aadhavan (SANDYS)   
2               1       1      Aadhavan (SANDYS)  K.Siddharthan (COSMO)   
3               1       1  K.Siddharthan (COSMO)      Aadhavan (SANDYS)   
4               1       1      Aadhavan (SANDYS)  K.Siddharthan (COSMO)   

                SCOREDBY  SCORE   TIME  OPPONENT SCORE  Unnamed: 8  \
0  K.Siddharthan (COSMO)    0.0  08:32             0.0         NaN   
1      Aadhavan (SANDYS)    1.0  08:33             0.0         NaN   
2  K.Siddharthan (COSMO)    1.0  08:33             1.0         NaN   
3      Aadhavan (SANDYS)    2.0  08:33             1.0         NaN   
4      Aadhavan (SANDYS)    3.0  08:33             1.0         NaN   

   Unnamed: 9  Unnamed: 10  Unnamed: 11  Unnamed: 12  
0         NaN          NaN          NaN          NaN  
1         NaN     

In [ ]:
pip install opencv-python ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.1/910.1 kB 13.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
# Define source and destination paths
source = '/content/yolo11x.pt'  # Change this to your source file path
destination = '/content/drive/MyDrive/icorte'  # Change this to your desired destination path

# Move the file
shutil.copy(source, destination)


'/content/drive/MyDrive/icorte/yolo11x.pt'

In [ ]:
import cv2
from ultralytics import YOLO
model = YOLO("/content/yolo11x.pt")  # Ensure you have the correct path to the model
# Load your image (replace 'your_image.jpg' with your actual image path)
image_path = 'your_image.jpg'
img = cv2.imread(image_path)
